In [1]:
# Load libraries
import numpy as np
from tensorflow.keras.applications import vgg16
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D, Reshape, multiply, Add, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model, layers
from PIL import Image
import os

In [4]:


# Function to load and preprocess the dataset
def load_preprocessed_dataset(fname):
    # Load the dataset
    data = np.genfromtxt(fname, dtype=['|S19', '<f8', '|S4'], names=[
                         'path', 'probability', 'type'])
    image_fnames = np.char.decode(data['path'])
    probs = data['probability']
    types = np.char.decode(data['type'])

    # Define a function to preprocess the images
    def load_and_preprocess_image(fname):
        with Image.open(fname) as image:
            # Preprocess image for VGG-16
            image = image.convert('RGB')
            image = image.resize((224, 224))
            image = np.array(image)
            image = vgg16.preprocess_input(image)
            return image
    
    # Load and preprocess images
    dir = os.path.dirname(fname)
    images = np.array([load_and_preprocess_image(os.path.join(dir, fn))
                       for fn in image_fnames])
    
    # Convert probabilities to categorical labels
    labels = to_categorical(probs * 3, num_classes=4)
    
    return images, labels, types

# Load the dataset
images, labels, types = load_preprocessed_dataset('elpv-dataset/labels.csv')

# Split the dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Define SE Block for attention
def se_block(input_feature, ratio=8):
    """ Create a squeeze and excitation block """
    channel_axis = -1
    channel = input_feature.shape[channel_axis]

    se_feature = GlobalAveragePooling2D()(input_feature)
    se_feature = Reshape((1, 1, channel))(se_feature)
    se_feature = Dense(channel // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se_feature)
    se_feature = Dense(channel, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se_feature)

    se_feature = multiply([input_feature, se_feature])
    return se_feature

# Load the VGG-16 model without the top layer (include_top=False)
base_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Adding SE blocks to VGG
layer_outputs = [layer.output for layer in base_model.layers]
attention_layer_1 = se_block(layer_outputs[-2]) # Add attention to a layer near the end
attention_layer_2 = se_block(layer_outputs[-5]) # Add attention to another layer

# Combine these layers with the rest of the model
x = layers.Concatenate()([attention_layer_1, attention_layer_2])
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(4, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile and train the model as before
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/5
66/66 [==============================] - 912s 14s/step - loss: 9.5265 - accuracy: 0.5445 - val_loss: 0.9064 - val_accuracy: 0.6362
Epoch 2/5
66/66 [==============================] - 386s 6s/step - loss: 0.9816 - accuracy: 0.6460 - val_loss: 0.8596 - val_accuracy: 0.6667
Epoch 3/5
66/66 [==============================] - 393s 6s/step - loss: 0.9601 - accuracy: 0.6617 - val_loss: 0.9004 - val_accuracy: 0.6705
Epoch 4/5
66/66 [==============================] - 398s 6s/step - loss: 0.9302 - accuracy: 0.6656 - val_loss: 0.7767 - val_accuracy: 0.7029
Epoch 5/5
66/66 [==============================] - 391s 6s/step - loss: 0.8465 - accuracy: 0.6980 - val_loss: 0.7785 - val_accuracy: 0.7048
